# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [2]:
! git clone https://github.com/ds3001f25/wrangling_assignment.git

Cloning into 'wrangling_assignment'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 0), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 5.83 MiB | 11.17 MiB/s, done.


In [3]:
import numpy as np # Import the numpy package into your workspace
import matplotlib.pyplot as plt # Import matplotlib into your workspace
import pandas as pd  # Import the pandas package into your workspace

In [23]:
airbnb_df = pd.read_csv('wrangling_assignment/data/airbnb_hw.csv',low_memory=False)

In [24]:
price_list = airbnb_df['Price'].tolist()
cleaned_price_list = []
for each in price_list:
  if "," in each:
    comma = each.index(',')
    cleaned_each = each[:comma] + each[comma+1:]
    cleaned_price_list.append(int(cleaned_each))
  else:
      cleaned_price_list.append(int(each))
airbnb_df['Price'] = cleaned_price_list
# Since this is a numerical variable, I wanted to turn the values of the Price column into integers. However, since some
#of them have commas if the value is over 1000, I first had to get rid of the commas before turning them into integers

In [26]:
police_df = pd.read_csv('wrangling_assignment/data/mn_police_use_of_force.csv',low_memory=False)
police_df['subject_injury']

,subject_injury
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12920,NaN
12921,NaN
12922,NaN
12923,NaN


In [27]:
police_df['subject_injury'].unique()

array([nan, 'No', 'Yes'], dtype=object)

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)